<a href="https://colab.research.google.com/github/SohamK04/AI-Video-Summarizer/blob/main/AI%20Video%20Summarizer%20Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install the required dependencies
!pip uninstall whisper -y  # Uninstall any incorrect whisper package
!pip install openai-whisper  # Install OpenAI's Whisper
!pip install moviepy
!pip install transformers
!pip install scikit-learn  # Install scikit-learn for cosine similarity

import os
import torch
import json
import subprocess
import whisper  # OpenAI Whisper library
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from moviepy.editor import VideoFileClip
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to install missing dependencies
def install_if_missing(package, import_name=None):
    if import_name is None:
        import_name = package
    try:
        __import__(import_name)
    except ModuleNotFoundError:
        subprocess.check_call(["pip", "install", package])
        __import__(import_name)

install_if_missing("moviepy")
install_if_missing("whisper")

# Load Whisper model for transcription
whisper_model = whisper.load_model("small")

# Replace LaMini with BART (open model)
MODEL_NAME = "facebook/bart-large-cnn"  # An open model for summarization
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

# Extract audio from video and transcribe using Whisper
def extract_and_transcribe(video_path):
    audio_path = video_path.replace(".mp4", ".wav")
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path, codec='pcm_s16le')

    result = whisper_model.transcribe(audio_path)
    transcript = result["text"]

    os.remove(audio_path)  # Cleanup audio file
    return transcript

# Summarize transcript using BART model
def summarize_bart(transcript):
    inputs = tokenizer("summarize: " + transcript, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(**inputs, max_length=250)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Compute scores based on summary quality using cosine similarity
def compute_scores(original, summary):
    # Convert original and summary to token embeddings using the tokenizer
    original_tokens = tokenizer(original, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    summary_tokens = tokenizer(summary, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # Calculate embeddings for original and summary text
    original_embeddings = model.base_model.get_input_embeddings()(original_tokens.input_ids).detach().cpu().numpy()
    summary_embeddings = model.base_model.get_input_embeddings()(summary_tokens.input_ids).detach().cpu().numpy()

    # Average the token embeddings to get a single vector representation per sequence
    original_avg_embedding = original_embeddings.mean(axis=1)  # Average over tokens
    summary_avg_embedding = summary_embeddings.mean(axis=1)  # Average over tokens

    # Calculate cosine similarity between the averaged embeddings
    similarity = cosine_similarity(original_avg_embedding, summary_avg_embedding)

    # Formula to compute accuracy, loss, and hallucination
    accuracy = np.clip(similarity.mean(), 0, 1)  # Average cosine similarity is used as accuracy
    loss = 1 - accuracy  # Higher loss when accuracy is lower

    # Hallucination Score: If cosine similarity is low, more hallucination
    hallucination_score = 1 - accuracy  # Hallucination score is inversely related to similarity

    # Optional: More detailed hallucination detection based on unique token counts
    original_tokens_set = set(original.lower().split())
    summary_tokens_set = set(summary.lower().split())
    hallucinated_tokens = summary_tokens_set - original_tokens_set
    hallucination_score = len(hallucinated_tokens) / len(summary_tokens_set)  # Percentage of hallucinated tokens

    return accuracy, loss, hallucination_score

# Process uploaded videos from Google Drive
video_folder = "/content/drive/MyDrive/video"
video_paths = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_path in video_paths:
    print(f"Processing video: {video_path}")
    transcript = extract_and_transcribe(video_path)
    summary = summarize_bart(transcript)

    accuracy, loss, hallucination_score = compute_scores(transcript, summary)

    print(f"Summary for {video_path}:")
    print(summary)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {loss * 100:.2f}%")
    print(f"Hallucination Score: {hallucination_score * 100:.2f}%")
    print("-" * 40)


  checkpoint = torch.load(fp, map_location=device)



Processing video: /content/drive/MyDrive/video/-esJrBWj2d8.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/-esJrBWj2d8.wav


MoviePy - Done.
Summary for /content/drive/MyDrive/video/-esJrBWj2d8.mp4:
"This is Miffy's show. We just fed our cat dog food. It's kind of funny" "Miffy survived the night. She ate dog food and didn't get sick. Did you, Miff? You're pretty healthy" "I like it when she sticks her head in the final. It feels good"
Accuracy: 92.21%
Loss: 7.79%
Hallucination Score: 13.95%
----------------------------------------
Processing video: /content/drive/MyDrive/video/b626MiF1ew4.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/b626MiF1ew4.wav


MoviePy - Done.
Summary for /content/drive/MyDrive/video/b626MiF1ew4.mp4:
Parkour is a discipline of training your mind and body to overcome obstacles. It's a process of breaking down obstacles in your head and figuring out the different parts. You might be physically capable of doing a new jump, but at the same time there's a fear telling you, like, oh you're going to get hurt.
Accuracy: 94.43%
Loss: 5.57%
Hallucination Score: 0.00%
----------------------------------------
Processing video: /content/drive/MyDrive/video/91IHQYk1IQM.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/91IHQYk1IQM.wav


MoviePy - Done.
Summary for /content/drive/MyDrive/video/91IHQYk1IQM.mp4:
summarize:  Music Music Music. Music Music, Music Music and Music Music is a collection of some of the world's best-known songs. The songs are arranged in a variety of styles, from jazz to classical to hip-hop. For more information on the songs, visit: www.musik.org.
Accuracy: 52.47%
Loss: 47.53%
Hallucination Score: 96.88%
----------------------------------------
Processing video: /content/drive/MyDrive/video/37rzWOQsNIw.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/37rzWOQsNIw.wav


...

MoviePy - Done.
Summary for /content/drive/MyDrive/video/z_6gVvQb2d0.mp4:
"I'm a kicker. Do you want to get in front of me? To me? I'm alright. Are you serious? Better ride faster than that," he says. "That was so close! And all of them. Look at all their people. Yeah, a lot of them"
Accuracy: 80.40%
Loss: 19.60%
Hallucination Score: 16.22%
----------------------------------------
Processing video: /content/drive/MyDrive/video/xmEERLqJ2kU.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/xmEERLqJ2kU.wav


MoviePy - Done.
Summary for /content/drive/MyDrive/video/xmEERLqJ2kU.mp4:
"We all go through adversity. Be it something major that happens to your family or to you or something like that" "Don't get caught up in the play that just happened. Play the next play" "Keep playing, keep hammering, keep getting after it" "Let's keep marching forward"
Accuracy: 89.72%
Loss: 10.28%
Hallucination Score: 20.51%
----------------------------------------
Processing video: /content/drive/MyDrive/video/Yi4Ij2NM7U4.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/Yi4Ij2NM7U4.wav


MoviePy - Done.
Summary for /content/drive/MyDrive/video/Yi4Ij2NM7U4.mp4:
The first episode of Warman's Meals features Josh Peach and Tarn Ferguson. The pair make a spicy sausage sandwich using ketchup, onions and peppers. Peach: "I was thinking of myself the other day. ... Obviously, myself is being stubborn or Tarnerguson"
Accuracy: 85.79%
Loss: 14.21%
Hallucination Score: 27.78%
----------------------------------------
Processing video: /content/drive/MyDrive/video/xxdtq8mxegs.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/xxdtq8mxegs.wav


MoviePy - Done.
Summary for /content/drive/MyDrive/video/xxdtq8mxegs.mp4:
Dogs develop dirt and debris and discharge in their ears and it has to be taken out. We use solutions like the ear cleansing solution to help break up debris and dirt. The cleaning solution helps kill bacteria and yeast and can be used afterwards for regular cleaning. If there are multiple pets in the house, it would be a good idea to have multiple bottles.
Accuracy: 93.53%
Loss: 6.47%
Hallucination Score: 4.00%
----------------------------------------
Processing video: /content/drive/MyDrive/video/xwqBXPGE9pQ.mp4
MoviePy - Writing audio in /content/drive/MyDrive/video/xwqBXPGE9pQ.wav


MoviePy - Done.
Summary for /content/drive/MyDrive/video/xwqBXPGE9pQ.mp4:
We're taking smartphone, spaceship, technology on a motorcycle platform with the safety of a car. It self-balances and really increases your safety and your range and the excitement of riding a motorcycle. 200 miles per charge, it's a two-wheel drive, a wheel hub motor, electric vehicle. The prototype that we have now uses two I7s.
Accuracy: 93.44%
Loss: 6.56%
Hallucination Score: 0.00%
----------------------------------------
